<a href="https://colab.research.google.com/github/yds725/nuclear_plant_proj/blob/code_development/lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# 고용량 램으로 바꾸는 법
# 돌려줘서 한번 그 노트에 25기가 램으로 바꾸면 다음번에는 안해줘도 됨
a = []
while(1):
    a.append('1')

In [0]:
## data load v2 
## 간략한 전처리 실시 
import pandas as pd
import numpy as np

def data_loader_v2(file_name, folder='', train_label=None, event_time=10, nrows=60):
    file_id = int(file_name.split('.')[0]) # file id만 불러오기
    df = pd.read_csv(folder+file_name, index_col=0, nrows=nrows) # 파일 읽어오기
    df = df.replace('.*', 0, regex=True).fillna(0) # 모든 문자열과 NA값을 0으로 대체
    df = df.loc[event_time:] # event_time 이후의 row들만 가지고 오기
    df.index = np.repeat(file_id, len(df)) # row 인덱스를 file id로 덮어 씌우기 
    if type(train_label) != type(None):
        label = train_label.loc[file_id]['label'] 
        df['label'] = np.repeat(label, len(df)) #train set일 경우 라벨 추가하기 
    return df

In [0]:
## 필요한 modules 불러오기 
import os
import pandas as pd 
import numpy as np
import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool 
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
#from data_loader_v2 import data_loader_v2 # 자체적으로 만든 data loader version 2.0 ([데이콘 15회 대회] 데이터 설명 및 데이터 불러오기 영상 참조)

from sklearn.ensemble import RandomForestClassifier
import joblib # 모델을 저장하고 불러오는 역할

In [0]:
##Set path 
train_folder = '/content/drive/My Drive/Nuclear_SemiProj/output/train/'
test_folder = '/content/drive/My Drive/Nuclear_SemiProj/output/test/'
train_label_path = '/content/drive/My Drive/Nuclear_SemiProj/train_label.csv'

In [0]:
##Load Files
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [0]:
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

In [2]:
chunksize = 10000
tfr = pd.read_csv("/content/drive/My Drive/Nuclear_SemiProj/output/v2_train_data.csv", chunksize=chunksize, iterator=True)
b_v2_train = pd.concat(tfr, ignore_index=True)

print(b_v2_train)

chunksize = 10000
tfr = pd.read_csv("/content/drive/My Drive/Nuclear_SemiProj/output/test_id_1.csv", chunksize=chunksize, iterator=True)
test1 = pd.concat(tfr, ignore_index=True)

test1.set_index("id", inplace=True)

test1.index.name = None

print(test1)



# chunksize = 10000
# tfr = pd.read_csv("/content/drive/My Drive/Nuclear_SemiProj/output/B_train_except30.csv", chunksize=chunksize, iterator=True)
# b_train_except30 = pd.concat(tfr, ignore_index=True)

# print(b_train_except30)

           V0000     V0001     V0002  ...  V5119  V5120  label
0      30.474394  8.691177  8.714483  ...   85.4    0.0    110
1      30.470463  8.736521  8.682769  ...   85.4    0.0    110
2      30.465427  8.753559  8.663426  ...   85.4    0.0    110
3      30.458532  8.715056  8.714854  ...   85.4    0.0    110
4      30.475773  8.790241  8.735125  ...   85.4    0.0    110
...          ...       ...       ...  ...    ...    ...    ...
41395  30.476523  8.680374  8.689265  ...   85.4    0.0    156
41396  30.464995  8.818355  8.688116  ...   85.4    0.0    156
41397  30.483490  8.737560  8.706527  ...   85.4    0.0    156
41398  30.459609  8.696491  8.685300  ...   85.4    0.0    156
41399  30.463722  8.707559  8.710813  ...   85.4    0.0    156

[41400 rows x 5122 columns]
          V0000     V0001     V0002     V0003  ...  V5117     V5118  V5119  V5120
1000  30.465741  8.618514  8.705075  8.730912  ...    0.0  0.000004   85.4    0.0
1000  30.477302  8.642689  8.713423  8.732450  ... 

In [0]:
# # standardize data
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import Normalizer

#b_all_train_edit = b_all_train.drop(["time", "id"], axis=1)
#print(b_all_train_edit)

# 이것은 30번 째 csv를 바꾸고 나서다 ㅅㅂ 
X_train = b_v2_train.drop(['label'], axis=1)
y_train = b_v2_train['label']


##Train

In [5]:
print(X_train.values)
print(y_train.values)

X_train_2d = X_train.values
y_train_2d = y_train.values

[[ 3.04743936e+01  8.69117682e+00  8.71448332e+00 ...  1.42161969e-05
   8.54000000e+01  0.00000000e+00]
 [ 3.04704625e+01  8.73652123e+00  8.68276888e+00 ... -6.11445497e-06
   8.54000000e+01  0.00000000e+00]
 [ 3.04654269e+01  8.75355883e+00  8.66342622e+00 ... -1.81329056e-05
   8.54000000e+01  0.00000000e+00]
 ...
 [ 3.04834898e+01  8.73756033e+00  8.70652705e+00 ...  1.46329850e-06
   8.54000000e+01  0.00000000e+00]
 [ 3.04596087e+01  8.69649110e+00  8.68529996e+00 ...  1.54051243e-05
   8.54000000e+01  0.00000000e+00]
 [ 3.04637217e+01  8.70755909e+00  8.71081314e+00 ... -1.66465620e-05
   8.54000000e+01  0.00000000e+00]]
[110 110 110 ... 156 156 156]


In [4]:
print(X_train_2d)
print(y_train_2d)

NameError: ignored

In [0]:
!pip install lightgbm

In [6]:
#!pip uninstall xgboost # pip uninstall
!pip install xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl

In [0]:
param_x = {}
# softmax multi-class class.
#param_x['objective'] = 'multi:softmax'
param_x['tree_method'] = 'gpu_hist'
param_x['verbosity'] = 2

In [0]:
# import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [0]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X_train_2d, y_train_2d, test_size=test_size, random_state=seed)

In [8]:
print(X_train)
print(y_train)

           V0000     V0001     V0002  ...         V5118  V5119  V5120
0      30.474394  8.691177  8.714483  ...  1.421620e-05   85.4    0.0
1      30.470463  8.736521  8.682769  ... -6.114455e-06   85.4    0.0
2      30.465427  8.753559  8.663426  ... -1.813291e-05   85.4    0.0
3      30.458532  8.715056  8.714854  ... -5.745568e-07   85.4    0.0
4      30.475773  8.790241  8.735125  ...  8.437883e-06   85.4    0.0
...          ...       ...       ...  ...           ...    ...    ...
41395  30.476523  8.680374  8.689265  ... -1.973322e-05   85.4    0.0
41396  30.464995  8.818355  8.688116  ... -8.723241e-06   85.4    0.0
41397  30.483490  8.737560  8.706527  ...  1.463299e-06   85.4    0.0
41398  30.459609  8.696491  8.685300  ...  1.540512e-05   85.4    0.0
41399  30.463722  8.707559  8.710813  ... -1.664656e-05   85.4    0.0

[41400 rows x 5121 columns]
0        110
1        110
2        110
3        110
4        110
        ... 
41395    156
41396    156
41397    156
41398    156
4

In [0]:
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV

lgbm = lgb.LGBMClassifier(n_estimators=100, silent = False)

#lgb.Dataset()

In [9]:
model = XGBClassifier(tree_method = 'gpu_hist', verbosity=2, max_depth = 6, n_jobs=-1) 
model.fit(X_train, y_train)

print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=-1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1, tree_method='gpu_hist',
              verbosity=2)


In [0]:
# params = {'max_depth': [10,15,20],
#           'min_child_samples': [20,40,60],
#           'subsample': [0.8, 1]}

# grid = GridSearchCV(lgbm, param_grid=params, scoring='roc_auc')

# lgb_model1 = grid.fit(X_train, y_train)

# print(lgb_model1.grid_scores_)
# print(lgb_model1.best_params_)
# print(lgb_model1.best_score_)

#grid.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_train, y_train), (X_val, y_val)] )

#print("최적 파라미터:", grid.best_params_ )

#lgbm_roc_score = roc_auc_score(Y_test, grid.predict_proba(X_test)[:, 1], average='macro')
#print("ROC AUC: {0:.4f}".format(lgbm_roc_score))

KeyboardInterrupt: ignored

In [0]:
# model_lgbm = lgb.LGBMClassifier(learning_rate=0.5, 
#                                 n_estimators=100, 
#                                 random_state=42, 
#                                 silent=False, 
#                                 verbose = 1,
#                                 max_depth = 6)

# model_lgbm.fit(X_train, y_train)


KeyboardInterrupt: ignored

In [10]:
print(test1)

          V0000     V0001     V0002     V0003  ...  V5117     V5118  V5119  V5120
1000  30.465741  8.618514  8.705075  8.730912  ...    0.0  0.000004   85.4    0.0
1000  30.477302  8.642689  8.713423  8.732450  ...    0.0  0.000005   85.4    0.0
1000  30.478336  8.675928  8.729837  8.672877  ...    0.0  0.000016   85.4    0.0
1000  30.462904  8.733765  8.706455  8.691974  ...    0.0 -0.000010   85.4    0.0
1000  30.483675  8.807382  8.680733  8.713651  ...    0.0 -0.000008   85.4    0.0
...         ...       ...       ...       ...  ...    ...       ...    ...    ...
999   30.462765  8.636107  8.717540  8.714714  ...    0.0 -0.000002   85.4    0.0
999   30.486345  8.687652  8.662559  8.723882  ...    0.0 -0.000005   85.4    0.0
999   30.471584  8.658865  8.692842  8.704296  ...    0.0  0.000006   85.4    0.0
999   30.464907  8.784168  8.716839  8.723631  ...    0.0  0.000003   85.4    0.0
999   30.476896  8.730331  8.695446  8.731110  ...    0.0  0.000004   85.4    0.0

[36000 rows x 5

In [0]:
#pred = model_lgbm.predict_proba(test1)
pred = model.predict_proba(test1)


In [0]:
# 제출파일 만들기 
submission = pd.DataFrame(data=pred)
submission.index = test1.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()
submission.to_csv('/content/drive/My Drive/submission/submission_xgboost.csv', index=True) #제출 파일 만들기